This is my "sandbox" to play around with the BEA api. My plan is to use this for my regional cycle project. Below I explain how to use the BEA api (stuff has changed since I last messed around with it)

In [187]:
import pandas as pd
import requests
import numpy as np

In [10]:
BEA_ID = "put in here"

After getting your key. [Here is some basic documentation](https://www.bea.gov/API/bea_web_service_api_user_guide.htm). Like a lot APIs, basic idea is to specify the URL in the proper way and it will return some output in a specified format. We can then convert the format (its .json) into a dataframe:



In [147]:
API_URL = "https://bea.gov/api/data?&UserID=6BF79D8C-8042-4196-88DC-0E0C55B0C3B6&method=GETDATASETLIST&ResultFormat=JSON&"
    
r = requests.get(API_URL)

In [152]:
type(r)


requests.models.Response

In [153]:
type(r.json())

dict

In [160]:
print(r.json().keys())

print(r.json()['BEAAPI'].keys())

print(r.json()['BEAAPI']["Results"].keys())

dict_keys(['BEAAPI'])
dict_keys(['Results', 'Request'])
dict_keys(['Dataset'])


This then leads to the insight that the `.json()` is a bunch of dictionaries within the dictionaries and that by working through it we can find a "root dictionary" that can be converted into a usable DataFrame

In [161]:
df = pd.DataFrame(r.json()["BEAAPI"]["Results"]['Dataset'])

In [162]:
df

,DatasetDescription,DatasetName
0,The RegionalData dataset is obsolete. Please u...,RegionalData
1,Standard NIPA tables,NIPA
2,Standard NI underlying detail tables,NIUnderlyingDetail
3,Multinational Enterprises,MNE
4,Standard Fixed Assets tables,FixedAssets
5,International Transactions Accounts,ITA
6,International Investment Position,IIP
7,GDP by Industry,GDPbyIndustry
8,Regional Income data sets,RegionalIncome
9,Regional Product data sets,RegionalProduct


Then this gives us the different datasets that are available through the BEA api. Now below, I'm going to grab personal income at the county level. The documentation for grabing this is [here](https://www.bea.gov/API/bea_web_service_api_user_guide.htm)

In [429]:
my_key = "https://bea.gov/api/data?&UserID=6BF79D8C-8042-4196-88DC-0E0C55B0C3B6&method=GetData&"

data_set = "datasetname=RegionalIncome&"

table_and_line_income = "TableName=CA1&LineCode=3&"

table_and_line_population = "TableName=CA1&LineCode=2&"

year = "Year=1969,2015&"

location = "GeoFips=COUNTY&"

form = "ResultFormat=json"

In [430]:
API_URL = my_key + data_set + table_and_line_income + year + location + form

r = requests.get(API_URL)

df_income = pd.DataFrame(r.json()["BEAAPI"]["Results"]["Data"])

In [431]:
df_income.drop(['CL_UNIT', 'Code',"NoteRef", "UNIT_MULT"], axis=1, inplace = True)

#df["DataValue"].column = "IncomePC"

df_income.rename(columns={"DataValue":"IncomePC"}, inplace=True)

df_income.head()

,IncomePC,GeoFips,GeoName,TimePeriod
0,3930,00000,United States,1969
1,48451,00000,United States,2015
2,38214,01000,Alabama,2015
3,2827,01000,Alabama,1969
4,2774,01001,"Autauga, AL",1969


In [432]:
API_URL = my_key + data_set + table_and_line_population + year + location + form

r = requests.get(API_URL)

population = pd.DataFrame(r.json()["BEAAPI"]["Results"]["Data"])

population.drop(['CL_UNIT', 'Code',"NoteRef", "UNIT_MULT", "GeoName"], axis=1, inplace = True)

#df["DataValue"].column = "IncomePC"

population.rename(columns={"DataValue":"Population"}, inplace=True)

population.head()

,Population,GeoFips,TimePeriod
0,201298000,00000,1969
1,320896618,00000,2015
2,4853875,01000,2015
3,3440000,01000,1969
4,25166,01001,1969


In [457]:
combo = pd.merge(population, df_income,   # left df, right df
                 how='inner',      # Try the different options, inner, outer, left, right...what happens.
                 on=['GeoFips',"TimePeriod"],       # link with cntry
                 indicator=True)  # Tells us what happend

In [458]:
combo["TimePeriod"] = pd.to_datetime(combo["TimePeriod"], infer_datetime_format = True)

#combo.sort_values(by = "TimePeriod", inplace = True)

In [459]:
combo.set_index(["GeoFips","TimePeriod"],inplace = True)

In [462]:
combo.sort_index(level="GeoFips", inplace = True)

In [464]:
combo.head()

Population IncomePC        GeoName _merge
GeoFips TimePeriod                                          
00000   1969-01-01  201298000     3930  United States   both
        2015-01-01  320896618    48451  United States   both
01000   1969-01-01    3440000     2827        Alabama   both
        2015-01-01    4853875    38214        Alabama   both
01001   1969-01-01      25166     2774    Autauga, AL   both

In [288]:
df.DataValue.replace("(NA)", np.nan, inplace = True)

In [289]:
df.DataValue = df.DataValue.astype(float)

In [290]:
df.dtypes

CL_UNIT        object
Code           object
DataValue     float64
GeoFips        object
GeoName        object
NoteRef        object
TimePeriod     object
UNIT_MULT      object
dtype: object

In [291]:
df["TimePeriod"] = pd.to_datetime(df["TimePeriod"], infer_datetime_format = True)

In [292]:
df.sort_values(by = "TimePeriod", inplace = True)


In [294]:
df.head()

,CL_UNIT,Code,DataValue,GeoFips,GeoName,NoteRef,TimePeriod,UNIT_MULT
0,dollars,CA1-3,3930.0,00000,United States,NaN,1969-01-01,0
3535,dollars,CA1-3,2800.0,31123,"Morrill, NE",NaN,1969-01-01,0
3536,dollars,CA1-3,3093.0,31125,"Nance, NE",NaN,1969-01-01,0
3539,dollars,CA1-3,3727.0,31127,"Nemaha, NE",NaN,1969-01-01,0
3540,dollars,CA1-3,3106.0,31129,"Nuckolls, NE",NaN,1969-01-01,0


In [346]:
grp = df.groupby("GeoFips")

In [310]:
def log_diff_income(df):
    df["growth"] = np.log(df.DataValue).diff() / 46
    
    return df

In [388]:
transform_dict = {"DataValue": ["first","last", lambda x: np.diff(np.log(x))/46]}

new_df = grp.agg(transform_dict)

In [394]:
new_df["DataValue"]["last"].corr(new_df["DataValue"]["<lambda>"])

0.414791642511866

In [367]:
test =  new_df.diff(axis = 1).toseries

AttributeError: 'DataFrame' object has no attribute 'toseries'

In [385]:
np.diff?

In [309]:
grp = df.groupby("GeoFips")
grp.get_group("44007").TimePeriod.diff()

Object `TimePeriod.diff` not found.


In [ ]:
grp.get_group("44007").TimePeriod.diff

In [ ]:
grp.get_group("44007").sort_value

In [ ]:
grp.get_group("44007").sort_values

In [277]:
new_df.sort_values?

In [332]:
new_df.growth.corr(new_df.loc["1969-01-01"].DataValue)

nan

In [325]:
new_df.set_index("TimePeriod", inplace = True)

In [376]:
df.diff?